In [1]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation,\
Configuration, Action, row_col, adjacent_positions, translate, min_distance,random_agent, GreedyAgent

from kaggle_environments import make
import numpy as np
from random import choice

import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
from collections import deque
import time

Loading environment football failed: No module named 'gfootball'


In [26]:
ACTIONS = ['NORTH', 'SOUTH', 'WEST', 'EAST']
env = make("hungry_geese")
display(env.reset())

trainer = env.train([None, "greedy", "greedy", "greedy"])
trainer.reset()

def DQNet():
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(7,11,17)))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,5), activation='LeakyReLU'))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='LeakyReLU'))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='LeakyReLU'))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='tahn'))
    
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    
    return model

model = DQNet()

def centerize(b):
    dy, dx = np.where(b[0])
    centerize_y = (np.arange(0,7)-3+dy[0])%7
    centerize_x = (np.arange(0,11)-5+dx[0])%11
    
    b = b[:, centerize_y,:]
    b = b[:, :,centerize_x]
    
    return b

def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1
        
    b = b.reshape(-1, 7, 11)
    b = centerize(b)
    b = np.transpose(b, (1,2,0))

    return b

def train(env, replay_memory, model, target_model, done):
    
    learning_rate = 0.7 # Learning rate
    discount_factor = 0.618
    batch_size = 512 * 2

    MIN_REPLAY_SIZE = 3000
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    
    mini_batch = random.sample(replay_memory, batch_size)
    
    rewards = np.array([transition[2] for transition in mini_batch])
    
    current_states = np.array([transition[0] for transition in mini_batch])
    current_qs_list = model.predict(current_states)
    
    
    new_current_states = np.array([transition[3] for transition in mini_batch])
    
#     non_final_states = np.array([transition[3] for transition in mini_batch if not transition[4]])
#     non_final_index_mask = np.array([False if transition[4] else True  for transition in mini_batch])
    
    
#     future_qs_list = tf.zeros(batch_size)
    
    future_qs_list = target_model.predict(new_current_states)
    
#     next_state_reward = future_qs_list*discount_factor + rewards

    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done) in enumerate(mini_batch):
        if not done:
            max_future_q = reward + discount_factor * np.max(future_qs_list[index])
        else:
            max_future_q = reward

#         current_qs = current_qs_list[index]
        current_qs = [0]*4
        current_qs[action] = (1 - learning_rate) * current_qs[action] + learning_rate * max_future_q

        X.append(observation)
        Y.append(current_qs)
#     print(Y)
    model.fit(np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True)
    

def main():
    trainer = env.train([None, "greedy", "greedy", "greedy"])
    trainer.reset()
    is_render = False
    
    epsilon = 1 # Epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
    max_epsilon = 1 # You can't explore more than 100% of the time
    min_epsilon = 0.01 # At a minimum, we'll always explore 1% of the time
    decay = 0.01
    
    # An episode a full game
    train_episodes = 1000
    test_episodes = 100

    # 1. Initialize the Target and Main models
    # Main Model (updated every 4 steps)
    model = DQNet()
    # Target Model (updated every 100 steps)
    target_model = DQNet()
    target_model.set_weights(model.get_weights())

    replay_memory = deque(maxlen=10000)

    target_update_counter = 0

    # X = states, y = actions
    X = []
    y = []
    
    observation_list = []
    steps_to_update_target_model = 0

    for episode in range(train_episodes):
        total_training_rewards = 0
        observation = trainer.reset()
        observation_list = []
        
        done = False
        print("Runnig episode no ", episode)
        while not done:
            observation_list.append(observation)
            encoded_observation = make_input(observation_list)
            
            steps_to_update_target_model += 1
            if is_render:
                env.render(mode="ipython", width=500, height=450)

            random_number = np.random.rand()
            # 2. Explore using the Epsilon Greedy Exploration Strategy
            if random_number <= epsilon or len(replay_memory) < 5000:
                # Explore
                g_agent = GreedyAgent(Configuration({'rows': 7, 'columns': 11}))
                action = g_agent(Observation(observation))
                
                action = ACTIONS.index(action)
            else:
                # Exploit best known action
                # model dims are (batch, env.observation_space.n)
                encoded_reshaped = encoded_observation.reshape(-1,7,11,17)
                
                predicted = model.predict(encoded_reshaped)
                action = np.argmax(predicted)
#             print(action)
            new_observation, reward, done, info = trainer.step(ACTIONS[action])
            if done and reward == 0:
                reward = -1000
            new_encoded_observation = make_input(observation_list)
            replay_memory.append([encoded_observation, action, reward, new_encoded_observation, done])

            # 3. Update the Main Network using the Bellman Equation
            if steps_to_update_target_model % 4 == 0 or done:
#                 print("***********Train************")
                train(env, replay_memory, model, target_model, done)

            observation = new_observation
            total_training_rewards += reward

            if done:
                print('Total training rewards: {} after n steps = {} with final reward = {}'.format(total_training_rewards, episode, reward))
                total_training_rewards += 1

                if steps_to_update_target_model >= 100:
                    print('Copying main network weights to the target network weights')
                    target_model.set_weights(model.get_weights())
                    steps_to_update_target_model = 0
                break

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
#     env.close()
    model.save('baseline.h5')


[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[28]],
   'food': [10, 60],
   'index': 0},
  'status': 'ACTIVE'}]

In [27]:
main()

Runnig episode no  0
Total training rewards: -699 after n steps = 0 with final reward = -1000
Runnig episode no  1
Total training rewards: -699 after n steps = 1 with final reward = -1000
Runnig episode no  2
Total training rewards: 2304 after n steps = 2 with final reward = -1000
Runnig episode no  3
Total training rewards: -499 after n steps = 3 with final reward = -1000
Runnig episode no  4
Total training rewards: 6410 after n steps = 4 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  5
Total training rewards: 3208 after n steps = 5 with final reward = -1000
Runnig episode no  6
Total training rewards: 10014 after n steps = 6 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  7
Total training rewards: -799 after n steps = 7 with final reward = -1000
Runnig episode no  8
Total training rewards: -699 after n steps = 8 with final reward = -1000
Runnig episode no  9
Total training r

Total training rewards: 4107 after n steps = 72 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  73
Total training rewards: 7713 after n steps = 73 with final reward = 100
Runnig episode no  74
Total training rewards: 8412 after n steps = 74 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  75
Total training rewards: 5010 after n steps = 75 with final reward = -1000
Runnig episode no  76
Total training rewards: -499 after n steps = 76 with final reward = -1000
Runnig episode no  77
Total training rewards: 14413 after n steps = 77 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  78
Total training rewards: 4608 after n steps = 78 with final reward = -1000
Runnig episode no  79
Total training rewards: 3912 after n steps = 79 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  80
T

Total training rewards: 2808 after n steps = 141 with final reward = -1000
Runnig episode no  142
Total training rewards: -699 after n steps = 142 with final reward = -1000
Runnig episode no  143
Total training rewards: 12320 after n steps = 143 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  144
Total training rewards: 10218 after n steps = 144 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  145
Total training rewards: -799 after n steps = 145 with final reward = -1000
Runnig episode no  146
Total training rewards: 5915 after n steps = 146 with final reward = -1000
Runnig episode no  147
Total training rewards: 6709 after n steps = 147 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  148
Total training rewards: 9113 after n steps = 148 with final reward = -1000
Copying main network weights to the target network weights
Run

Total training rewards: 9217 after n steps = 208 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  209
Total training rewards: -699 after n steps = 209 with final reward = -1000
Runnig episode no  210
Total training rewards: 2205 after n steps = 210 with final reward = -1000
Runnig episode no  211
Total training rewards: 5309 after n steps = 211 with final reward = -1000
Runnig episode no  212
Total training rewards: 6213 after n steps = 212 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  213
Total training rewards: 5012 after n steps = 213 with final reward = -1000
Runnig episode no  214
Total training rewards: -399 after n steps = 214 with final reward = -1000
Runnig episode no  215
Total training rewards: 7312 after n steps = 215 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  216
Total training rewards: 6712 after n step

Total training rewards: 202 after n steps = 277 with final reward = -1000
Runnig episode no  278
Total training rewards: 12220 after n steps = 278 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  279
Total training rewards: 3205 after n steps = 279 with final reward = -1000
Runnig episode no  280
Total training rewards: 9721 after n steps = 280 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  281
Total training rewards: 3407 after n steps = 281 with final reward = -1000
Runnig episode no  282
Total training rewards: -499 after n steps = 282 with final reward = -1000
Runnig episode no  283
Total training rewards: 7506 after n steps = 283 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  284
Total training rewards: 4607 after n steps = 284 with final reward = -1000
Runnig episode no  285
Total training rewards: 5409 after n steps = 

...

Total training rewards: 10814 after n steps = 735 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  736
Total training rewards: 4308 after n steps = 736 with final reward = -1000
Runnig episode no  737
Total training rewards: 7009 after n steps = 737 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  738
Total training rewards: 7010 after n steps = 738 with final reward = -1000
Runnig episode no  739
Total training rewards: 3107 after n steps = 739 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  740
Total training rewards: 2404 after n steps = 740 with final reward = -1000
Runnig episode no  741
Total training rewards: 3001 after n steps = 741 with final reward = -1000
Runnig episode no  742
Total training rewards: 5806 after n steps = 742 with final reward = -1000
Copying main network weights to the target network weights
Runn

Total training rewards: 4508 after n steps = 799 with final reward = -1000
Runnig episode no  800
Total training rewards: 5309 after n steps = 800 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  801
Total training rewards: 2205 after n steps = 801 with final reward = -1000
Runnig episode no  802
Total training rewards: 5510 after n steps = 802 with final reward = -1000
Runnig episode no  803
Total training rewards: 6815 after n steps = 803 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  804
Total training rewards: 4712 after n steps = 804 with final reward = -1000
Runnig episode no  805
Total training rewards: 7512 after n steps = 805 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  806
Total training rewards: 5208 after n steps = 806 with final reward = -1000
Runnig episode no  807
Total training rewards: 12313 after n ste

Total training rewards: 7311 after n steps = 864 with final reward = -1000
Runnig episode no  865
Total training rewards: 3001 after n steps = 865 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  866
Total training rewards: 9415 after n steps = 866 with final reward = 100
Runnig episode no  867
Total training rewards: 16226 after n steps = 867 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  868
Total training rewards: 13719 after n steps = 868 with final reward = 101
Copying main network weights to the target network weights
Runnig episode no  869
Total training rewards: 3305 after n steps = 869 with final reward = -1000
Runnig episode no  870
Total training rewards: 4203 after n steps = 870 with final reward = -1000
Runnig episode no  871
Total training rewards: 3106 after n steps = 871 with final reward = -1000
Copying main network weights to the target network weights
Runni

Total training rewards: 2905 after n steps = 929 with final reward = -1000
Runnig episode no  930
Total training rewards: 4806 after n steps = 930 with final reward = -1000
Runnig episode no  931
Total training rewards: 3606 after n steps = 931 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  932
Total training rewards: 2405 after n steps = 932 with final reward = -1000
Runnig episode no  933
Total training rewards: 6409 after n steps = 933 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  934
Total training rewards: 5012 after n steps = 934 with final reward = -1000
Runnig episode no  935
Total training rewards: 9213 after n steps = 935 with final reward = -1000
Copying main network weights to the target network weights
Runnig episode no  936
Total training rewards: 8115 after n steps = 936 with final reward = -1000
Runnig episode no  937
Total training rewards: 10417 after n s

Total training rewards: 4108 after n steps = 996 with final reward = -1000
Runnig episode no  997
Total training rewards: 902 after n steps = 997 with final reward = -1000
Runnig episode no  998
Total training rewards: 8713 after n steps = 998 with final reward = 100
Copying main network weights to the target network weights
Runnig episode no  999
Total training rewards: 11619 after n steps = 999 with final reward = -1000
Copying main network weights to the target network weights


In [2]:
obs_list = []
obs = trainer.reset()
obs_list.append(obs)
model = tf.keras.models.load_model('my_model.h5')

done = False
total_reward = 0
while not done:
    obs_list.append(obs)
    encoded_observation = make_input(obs_list)
    encoded_reshaped = encoded_observation.reshape(-1,7,11,17)
                
    predicted = model.predict(encoded_reshaped)
#     print(predicted)
    action = np.argmax(predicted)
#     print(action)
    new_obs, reward, done, info = trainer.step(ACTIONS[action])
#     env.render(mode="ipython", width=500, height=450)
    
    obs = new_obs
    total_reward += reward
    
    if done:
        print(f'Total reward: {total_reward}')


NameError: name 'trainer' is not defined

In [ ]:
# 10815

In [5]:
from kaggle_environments import make
env = make("hungry_geese", debug=True)

env.reset()
env.run(['main.py', 'main.py','main.py','main.py'])
env.render(mode="ipython", width=500, height=450)

Opposite action: (0, <Action.NORTH: 1>, <Action.SOUTH: 3>)
Goose Starved: Action.EAST
Goose Starved: Action.EAST
